In [1]:
#importing necessary libraries
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

#initialising Spark session    
spark = SparkSession  \
    .builder  \
    .appName("spark-streaming")  \
    .getOrCreate()
spark.sparkContext.setLogLevel('ERROR')

#reading input from Kafka
orderRawData = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "18.211.252.152:9092") \
    .option("startingOffsets","earliest") \
    .option("failOnDataLoss", "false") \
    .option("subscribe", "transactions-topic-verified") \
    .load()

#defining schema for a single order
jsonSchema = StructType() \
    .add("card_id", StringType()) \
    .add("member_id", StringType()) \
    .add("amount", LongType()) \
    .add("pos_id", StringType()) \
    .add("postcode", StringType()) \
    .add("transaction_dt", TimestampType()) 

   
#creating an order stream for reading data from json in kafka 
orderStream = orderRawData.select(from_json(col("value").cast("string"), jsonSchema).alias("data")).select("data.*")

query = orderStream  \
	.select("card_id", "member_id", "amount", "pos_id", "postcode", "transaction_dt") \
	.writeStream \
        .outputMode("append") \
        .format("console") \
        .option("truncate", "false") \
        .trigger(processingTime = "1 minute") \
        .start()

#indicating Spark to await termination
query.awaitTermination()


ModuleNotFoundError: No module named 'pyspark'

In [2]:
pip install pyspark

  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=e2c00deedba23ca0dfe1a9581c2f4150d80f63821978ebe345193b12e653be42
  Stored in directory: c:\users\dnkii\appdata\local\pip\cache\wheels\6c\e3\9b\0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.
